In [4]:
import scipy.stats as ss
import numpy as np

def LSM(S0, K, T, r, sig, payoff, N=45, paths=10000, order=2):

    dt = T/(N-1)          # time interval
    df = np.exp(r * dt)  # discount factor per time time interval

    X0 = np.zeros((paths,1))
    increments = ss.norm.rvs(loc=(r - sig**2/2)*dt, scale=np.sqrt(dt)*sig, size=(paths,N-1))
    X = np.concatenate((X0,increments), axis=1).cumsum(1)
    S = S0 * np.exp(X)
    if payoff == "put":
        H = np.maximum(K - S, 0)   # intrinsic values for put option
    if payoff == "call":
        H = np.maximum(S - K, 0)   # intrinsic values for call option
    V = np.zeros_like(H)            # value matrix
    V[:,-1] = H[:,-1]

    # Valuation by LS Method
    for t in range(N-2, 0, -1):
        good_paths = H[:,t] > 0    
        # polynomial regression：将EV>0的部分挑出来回归
        rg = np.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    
        # 估计E(HV)
        C = np.polyval( rg, S[good_paths,t] )                             
        # 如果E(HV)<EV，那么行权
        exercise = np.zeros( len(good_paths), dtype=bool)
        exercise[good_paths] = H[good_paths,t] > C
        V[exercise,t] = H[exercise,t]
        V[exercise,t+1:] = 0
        discount_path = (V[:,t] == 0)
        V[discount_path,t] = V[discount_path,t+1] * df
    V0 = np.mean(V[:,1]) * df  # 
    return V0

call = LSM(100, 100, 1, 0.1, 0.2, 'call', N=10000, paths=10000, order=2)
put = LSM(100, 100, 1, 0.1, 0.2, 'put', N=10000, paths=10000, order=2)

In [7]:
call, put

(15.842643964447301, 5.282024498539422)